# Méthode de Steihaug-Toint

In [1]:
using BenchmarkTools
using LinearAlgebra
using ForwardDiff

verbose = false

false

## Algorithme de région de confiance basique

Nous allons regrouper les constantes dans la structure `BasicTrustRegion`.

In [2]:
struct BasicTrustRegion{T <: Real}
    η1:: T
    η2:: T
    γ1:: T
    γ2:: T
end

function BTRDefaults()
    return BasicTrustRegion(0.01,0.9,0.5,0.5)
end

# We define a type to store the state at a given iteration.
mutable struct BTRState
    iter::Int
    x::Vector
    xcand::Vector
    g::Vector
    step::Vector
    Δ::Float64
    ρ::Float64

    tol::Float64

    trace
    keepTrace::Bool
    
    function BTRState()
        state = new()
        state.tol = 1e-6
        state.keepTrace = false
        return state
    end
end

In [3]:
function acceptCandidate!(state::BTRState, b::BasicTrustRegion)
    # If the iteration is successful, update the iterate
    if (state.ρ >= b.η1)
        return true
    else
        return false
    end
end

acceptCandidate! (generic function with 1 method)

In [4]:
function updateRadius!(state::BTRState, b::BasicTrustRegion)
    if (state.ρ >= b.η2)
        stepnorm = norm(state.step)
        state.Δ = min(1e20,max(4*stepnorm,state.Δ))
    elseif (state.ρ >= b.η1)
        state.Δ *= b.γ2
    else
        state.Δ *= b.γ1
    end
end

updateRadius! (generic function with 1 method)

## Algorithme de région de confiance de base

In [5]:
function btr(f::Function, ∇f::Function, ∇2f::Function, Step::Function,
             x0::Vector, state:: BTRState = BTRState(), ApproxH:: Bool = false, verbose::Bool = false)
    
    b = BTRDefaults()
    state.iter = 0
    state.x = x0
    n = length(x0)

    tol2 = state.tol*state.tol
    
    state.g = zeros(n)
    
    fx = f(x0)
    state.g = ∇f(x0)
    state.Δ = 0.1*norm(state.g) # 1.0

    if (ApproxH)
        # A better initialization procedure should be used with quasi-Newton approximations
        # We could rely on some preconditioner.
        H = zeros(n,n)+I
        y = zeros(n)
        gcand = zeros(n)
    else
        H = ∇2f(x0)
    end
    
    nmax = 1000
    if (state.keepTrace)
        state.trace= x0'
    end
    
    function model(s:: AbstractVector{T}, g:: AbstractVector{T}, H:: AbstractMatrix{T}) where T
        return dot(s, g)+0.5*dot(s, H*s)
    end
    
    while (dot(state.g,state.g) > tol2 && state.iter < nmax)
        # Compute the step by approximately minimize the model
        state.step = Step(state.g, H, state.Δ)
        state.xcand = state.x+state.step
        
        # Compute the actual reduction over the predicted reduction
        fcand = f(state.xcand)
        state.ρ = (fcand-fx)/(model(state.step, state.g, H))

       if (ApproxH)
            gcand = ∇f(state.xcand)
            y = gcand-state.g
            sy = dot(state.step,y)
#            if (sy < 1e-6)
#                println(state.iter, ". ", state.ρ, " ", state.Δ, " ", norm(state.step), " ", (model(state.step, state.g, H)), " ", norm(y), " ", sy, " ", norm(state.g))
#            end
            H = ∇2f(H, y, state.step)
        end

        if (acceptCandidate!(state, b))
            state.x = copy(state.xcand)
            if (ApproxH == false)
                state.g = ∇f(state.x)
                H = ∇2f(state.x)
            else
                state.g = copy(gcand)
            end
            fx = fcand
        end

        if (state.keepTrace)
            state.trace= [state.trace ; state.x']
        end
        
        updateRadius!(state, b)
        state.iter += 1
    end
    
    return state
end

btr (generic function with 4 methods)

In [6]:
function CauchyStep(g::Vector, H::Matrix, Δ::Float64)
    q = dot(g,H*g)
    normg = norm(g)
    if (q <= 0)
        τ = 1.0
    else
        τ = min((normg*normg*normg)/(q*Δ),1.0)
    end
    return -τ*g*Δ/normg
end

CauchyStep (generic function with 1 method)

In [7]:
function BFGSUpdate(B:: Matrix, y:: Vector, s:: Vector)
    sy = dot(s,y)
    if (sy > 1e-12)
        Bs = B*s
        B[:,:] = B - (Bs*Bs')/dot(s, Bs) + (y*y')/sy
    end
    
    return B
end

BFGSUpdate (generic function with 1 method)

In [8]:
function SR1Update(B:: Matrix, y:: Vector, s:: Vector)
    Bs = B*s
    z = y-Bs
    den = dot(z,s)
    if (abs(den) > 1e-8)
        B[:,:] = B + (z*z')/den
    end
    
    return B
end

SR1Update (generic function with 1 method)

## Gradient conjugué tronqué

In [9]:
function stopCG(normg::Float64, normg0::Float64, k::Int, kmax::Int, χ::Float64 = 0.1, θ::Float64 = 0.5)
    if ((k == kmax) || (normg <= normg0*min(χ, normg0^θ)))
        if (verbose)
            println("CG stops after $k iterations")
        end
        return true
    else
        return false
    end
end

stopCG (generic function with 3 methods)

In [10]:
function TruncatedCG(g::AbstractVector{T}, H::AbstractMatrix{T}, Δ::Float64) where T
    n = length(g)
    s = zeros(n)

    normg0 = norm(g)
    v = g
    d = -v
    gv = dot(g,v)
    norm2d = gv
    norm2s = 0
    sMd = 0
    k = 0
    Δ2 = Δ*Δ

    while (stopCG(norm(g), normg0, k, n) == false)
        Hd = H*d  # we can redefine the multiplication operator
        κ = dot(d,Hd)
 
        # Is the curvature negative in the direction d?
        if (κ <= 0)
            if (k == 0)
                s = d/norm(d)*Δ
            else
                σ = (-sMd+sqrt(sMd*sMd+norm2d*(Δ2-dot(s,s))))/norm2d
#                σ = (-sMd+sqrt(sMd*sMd+norm2d*(Δ2-norm2s)))/norm2d
                s += σ*d
            end
            break
        end

        # κ > 0 -- the curvature is positive
        α = gv/κ

        # Check is the model minimizer is outside the trust region
        norm2s += α*(2*sMd+α*norm2d)
        if (norm2s >= Δ2)
            if (k == 0)
                s = d/norm(d)*Δ
            else
                σ = (-sMd+sqrt(sMd*sMd+norm2d*(Δ2-dot(s,s))))/norm2d
                # σ = (-sMd+sqrt(sMd*sMd+norm2d*(Δ2-oldnorm2s)))/norm2d
                s += σ*d
            end
            break
        end

        # The model minimizer is inside the trust region
        s += α*d
        g += α*Hd
        v = g
        newgv = dot(g,v)
        β = newgv/gv
        gv = newgv
        d = -v+β*d
        
        sMd = β*(sMd+α*norm2d)
        norm2d = gv+β*β*norm2d
        
        k += 1;
    end
    
    return s
end

TruncatedCG (generic function with 1 method)

## Exemple

In [ ]:
function rosenbrock(x::Vector)
    return (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
end

function rosenbrock_gradient(x::Vector)
    return [ (-2.0 * (1.0 - x[1]) - 400.0 * (x[2] - x[1]^2) * x[1])
             (200.0 * (x[2] - x[1]^2)) ]
end

function rosenbrock_hessian(x::Vector)
    return [ (2.0 - 400.0 * x[2] + 1200.0 * x[1]^2) (-400.0 * x[1]) ;
             (-400.0 * x[1]) 200.0 ]
end

function rosenbrock_gradient!(x::Vector, storage::Vector)
    storage[1] = -2.0 * (1.0 - x[1]) - 400.0 * (x[2] - x[1]^2) * x[1]
    storage[2] = 200.0 * (x[2] - x[1]^2)
end

function rosenbrock_hessian!(x::Vector, storage::Matrix)
    storage[1, 1] = 2.0 - 400.0 * x[2] + 1200.0 * x[1]^2
    storage[1, 2] = -400.0 * x[1]
    storage[2, 1] = -400.0 * x[1]
    storage[2, 2] = 200.0
end

In [ ]:
defaultState = BTRState()

In [ ]:
defaultState.tol

In [ ]:
state = btr(rosenbrock, rosenbrock_gradient, rosenbrock_hessian, CauchyStep, [0,0])

In [ ]:
state = btr(rosenbrock, rosenbrock_gradient, rosenbrock_hessian, TruncatedCG, [0,0])

In [ ]:
defaultState.tol = 1e-4

In [ ]:
state = btr(rosenbrock, rosenbrock_gradient, rosenbrock_hessian, CauchyStep, [0,0], defaultState)

In [ ]:
state = btr(rosenbrock, rosenbrock_gradient, rosenbrock_hessian, TruncatedCG, [0,0], defaultState)

In [ ]:
@benchmark btr(rosenbrock, rosenbrock_gradient, rosenbrock_hessian, CauchyStep, [0,0], defaultState)

In [ ]:
@benchmark btr(rosenbrock, rosenbrock_gradient, rosenbrock_hessian, TruncatedCG, [0,0], defaultState)

In [ ]:
state = btr(rosenbrock, rosenbrock_gradient, BFGSUpdate, TruncatedCG, [0,0], defaultState, true)

In [ ]:
@benchmark btr(rosenbrock, rosenbrock_gradient, BFGSUpdate, TruncatedCG, [0,0], defaultState, true)

In [ ]:
state = btr(rosenbrock, rosenbrock_gradient, SR1Update, TruncatedCG, [0,0], defaultState, true)

In [ ]:
@benchmark btr(rosenbrock, rosenbrock_gradient, SR1Update, TruncatedCG, [0,0], defaultState, true)

"Trust-Region Methods", Introduction
$$
f(x,y) = -10x^2+10y^2+4\sin(xy)-2x+x^4
$$

In [ ]:
using ForwardDiff
defaultState.tol = 1e-6

In [ ]:
f(x::Vector) = -10*x[1]^2+10*x[2]^2+4*sin(x[1]*x[2])-2*x[1]+x[1]^4

In [ ]:
g = x -> ForwardDiff.gradient(f, x);
H = x -> ForwardDiff.hessian(f, x)

function g!(x::Vector, storage::Vector)
    s = g(x)
    storage[1:length(s)] = s[1:length(s)]
end

In [ ]:
function H!(x::Vector, storage::Matrix)
    s = H(x)
    n, m = size(s)
    storage[1:n,1:m] = s[1:length(s)]
end

In [ ]:
state = btr(f, g, H, CauchyStep, [0,0])

In [ ]:
state = btr(f, g, H, TruncatedCG, [0,0])

In [ ]:
state = btr(f, g, BFGSUpdate, CauchyStep, [0,0], BTRState(), true)

In [ ]:
state = btr(f, g, BFGSUpdate, TruncatedCG, [0,0], BTRState(), true)

In [ ]:
f([2.30663, -0.332309])

In [ ]:
state = btr(f, g, H, TruncatedCG, [-2.0,-2.0], BTRState())

In [ ]:
f([-2.21022, 0.329748])

In [ ]:
st = BTRState()
st.keepTrace = true

In [ ]:
state = btr(f, g, BFGSUpdate, TruncatedCG, [-2.0,-2.0], st, true)

In [ ]:
st.trace

## Rosenbrock généralisé

Les exemples précédents travaillent sur des fonctions à deux dimensions, limitant la capacité d'évaluer les gains du gradient conjugué tronqué. Reprenons la fonction de Rosenbrock généralisée
$$
f(x) = \sum_{i = 1}^{n-1} \left( 100(x_{i+1}^2-x_i)^2 + (x_i-1)^2 \right).
$$
avec l'implémentation du gradient et la matrice Hessienne, avec un stockage dense ou creux.

In [11]:
function rosenbrock(x::Vector)
    return sum(100*(x[i+1]^2 - x[i])^2 + (x[i] - 1)^2 for i in 1:length(x)-1)
end

function ∇f(x:: Vector)
    n = length(x)
    g = zeros(n)
    for i = 1:n-1
        g[i] = -200*(x[i+1]^2-x[i])+2*(x[i]-1)
    end
    for i = 2:n
        g[i] += 400*x[i]*(x[i]^2-x[i-1])
    end
    return g
end

function Hess(x:: Vector)
    n = length(x)
    H = zeros(n,n)
    H[1,1] = 202
    for i = 2:n
        H[i,i-1] = H[i-1,i] = -400*x[i]
        H[i,i] = 400*(x[i]^2-x[i-1])+800*x[i]^2 + 202
    end
    H[n,n] -= 202
    return H
end

function TriHess(x)
    n = length(x)
    d = zeros(n)
    d[1] = 202
    d[2:n] = [400*(x[i]^2-x[i-1])+800*x[i]^2 + 202 for i = 2:n]
    d[n] -= 202
    dl = [-400*x[i] for i = 2:n]
    H = SymTridiagonal(d, dl)
    return H
end

TriHess (generic function with 1 method)

In [12]:
function ∇f!(x:: Vector, g:: Vector)
    g[:] = ∇f(x)
    return g
end

function Hess!(x:: Vector, H:: Matrix)
    H[:,:] = Hess(x)
    return H
end

function TriHess!(x:: AbstractVector{T}, H:: AbstractMatrix{T}) where T
    n = length(x)
    d = zeros(n)
    d[1] = 202
    d[2:n] = [400*(x[i]^2-x[i-1])+800*x[i]^2 + 202 for i = 2:n]
    d[n] -= 202
    dl = [-400*x[i] for i = 2:n]
    H = SymTridiagonal(d, dl)
    return H
end

TriHess! (generic function with 1 method)

In [13]:
gr = x -> ForwardDiff.gradient(rosenbrock, x);
Hr = x -> ForwardDiff.hessian(rosenbrock, x)

#14 (generic function with 1 method)

In [ ]:
n = 5

x = 10*ones(n)
x[1] = 2
x[5] = 5
g = zeros(n)
H = zeros(n,n)

∇f!(x, g)
H1 = Hess!(x, H)

H = zeros(2,2)
H2 = TriHess!(x, H)

norm(H1-H2)

In [ ]:
state = btr(rosenbrock, ∇f, TriHess, TruncatedCG, x, BTRState())

In [ ]:
state = btr(rosenbrock, ∇f, Hess, TruncatedCG, x, BTRState())

In [ ]:
state.x

In [ ]:
state.g

In [ ]:
state = btr(rosenbrock, gr, Hr, TruncatedCG, x, BTRState())

In [23]:
n = 1000

x = 10*ones(n)

1000-element Array{Float64,1}:
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
  ⋮
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0
 10.0

In [24]:
@benchmark btr(rosenbrock, ∇f, Hess, TruncatedCG, x, BTRState())

BenchmarkTools.Trial: 
  memory estimate:  220.32 MiB
  allocs estimate:  3377
  --------------
  minimum time:     478.532 ms (4.83% GC)
  median time:      955.527 ms (7.68% GC)
  mean time:        873.954 ms (12.44% GC)
  maximum time:     1.274 s (6.44% GC)
  --------------
  samples:          7
  evals/sample:     1

In [25]:
@benchmark btr(rosenbrock, ∇f, TriHess, TruncatedCG, x, BTRState())

BenchmarkTools.Trial: 
  memory estimate:  22.56 MiB
  allocs estimate:  3472
  --------------
  minimum time:     6.086 ms (0.00% GC)
  median time:      20.499 ms (18.34% GC)
  mean time:        25.787 ms (16.88% GC)
  maximum time:     154.513 ms (43.62% GC)
  --------------
  samples:          194
  evals/sample:     1

In [ ]:
@benchmark btr(rosenbrock, gr, Hr, TruncatedCG, x, BTRState())

In [ ]:
verbose = true
btr(rosenbrock, ∇f, Hess, TruncatedCG, x, BTRState())

In [ ]:
verbose = false
btr(rosenbrock, ∇f, TriHess, TruncatedCG, x, BTRState())

In [26]:
verbose = false
n = 20

x = 5*ones(n)

st = BTRState()
state = btr(rosenbrock, ∇f, BFGSUpdate, TruncatedCG, x, st, true)

BTRState(141, [0.9999999985869749, 0.9999999993548436, 0.9999999996249591, 0.9999999993543301, 0.9999999996215365, 0.999999999821721, 0.9999999999449956, 0.999999999989296, 1.0000000000309988, 1.000000000000124, 0.9999999999737147, 0.9999999999884466, 1.000000000019734, 1.0000000000013136, 1.0000000000049618, 1.0000000000152072, 1.0000000000230216, 0.9999999999486845, 0.9999999999501831, 0.9999999999592727], [0.9999999985869749, 0.9999999993548436, 0.9999999996249591, 0.9999999993543301, 0.9999999996215365, 0.999999999821721, 0.9999999999449956, 0.999999999989296, 1.0000000000309988, 1.000000000000124, 0.9999999999737147, 0.9999999999884466, 1.000000000019734, 1.0000000000013136, 1.0000000000049618, 1.0000000000152072, 1.0000000000230216, 0.9999999999486845, 0.9999999999501831, 0.9999999999592727], [-2.7368529442739487e-8, 6.877972588301305e-8, 1.4053952048140617e-7, -3.455594423793996e-7, -4.9640890492452176e-8, -5.248333769144503e-9, 2.047872182791795e-8, -1.1231267028950116e-9, 3.52

In [15]:
norm(state.g)

8.975453014788778e-7

In [27]:
state = btr(rosenbrock, ∇f, SR1Update, TruncatedCG, x, st, true)

BTRState(96, [0.9999999997242937, 0.9999999996877356, 0.9999999997923018, 1.000000000024184, 1.0000000000465294, 0.999999999895213, 0.9999999996788608, 0.9999999998726479, 1.0000000000732938, 1.0000000000730773, 1.0000000000422449, 1.0000000000928155, 1.0000000000774838, 1.0000000001234934, 1.0000000003950658, 1.0000000001388851, 1.0000000000991078, 1.0000000000260634, 0.9999999999811718, 0.9999999997315961], [0.9999999997242937, 0.9999999996877356, 0.9999999997923018, 1.000000000024184, 1.0000000000465294, 0.999999999895213, 0.9999999996788608, 0.9999999998726479, 1.0000000000732938, 1.0000000000730773, 1.0000000000422449, 1.0000000000928155, 1.0000000000774838, 1.0000000001234934, 1.0000000003950658, 1.0000000001388851, 1.0000000000991078, 1.0000000000260634, 0.9999999999811718, 0.9999999997315961], [6.921309259055874e-8, -1.1952714991187517e-7, -9.28814089998592e-8, 8.869985413077059e-8, 7.886369335480771e-8, 4.847380951821027e-9, -2.289258704133555e-7, -2.8468668759105437e-8, 9.515

In [28]:
btr(rosenbrock, ∇f, TriHess, TruncatedCG, x, st)

BTRState(40, [0.9999999999727507, 1.0000000000168785, 1.000000000061885, 1.00000000005114, 1.0000000000438944, 1.0000000000194906, 1.0000000000040106, 1.0000000000221276, 1.000000000019801, 1.0000000000111218, 1.0000000000100315, 1.0000000000000686, 0.9999999999909356, 0.9999999999602891, 0.9999999999417244, 1.0000000000020663, 1.0000000000504174, 1.0000000000109437, 0.9999999999686523, 0.9999999999726698], [0.9999999999727507, 1.0000000000168785, 1.000000000061885, 1.00000000005114, 1.0000000000438944, 1.0000000000194906, 1.0000000000040106, 1.0000000000221276, 1.000000000019801, 1.0000000000111218, 1.0000000000100315, 1.0000000000000686, 0.9999999999909356, 0.9999999999602891, 0.9999999999417244, 1.0000000000020663, 1.0000000000504174, 1.0000000000109437, 0.9999999999686523, 0.9999999999726698], [-1.22557608506213e-8, 3.0580045166883488e-9, 3.480131960651811e-8, 8.930507001397062e-9, 1.5729987535384004e-8, 3.676072779129609e-10, -1.262871496267756e-8, 1.2647240588493472e-8, 6.5408904

In [29]:
@benchmark btr(rosenbrock, ∇f, SR1Update, TruncatedCG, x, st, true)

BenchmarkTools.Trial: 
  memory estimate:  2.21 MiB
  allocs estimate:  7924
  --------------
  minimum time:     1.350 ms (0.00% GC)
  median time:      2.577 ms (0.00% GC)
  mean time:        3.813 ms (3.84% GC)
  maximum time:     89.308 ms (34.21% GC)
  --------------
  samples:          1311
  evals/sample:     1

In [30]:
@benchmark btr(rosenbrock, gr, Hr, TruncatedCG, x, st)

BenchmarkTools.Trial: 
  memory estimate:  1.51 MiB
  allocs estimate:  3591
  --------------
  minimum time:     1.708 ms (0.00% GC)
  median time:      2.349 ms (0.00% GC)
  mean time:        2.825 ms (2.81% GC)
  maximum time:     22.876 ms (0.00% GC)
  --------------
  samples:          1768
  evals/sample:     1

In [31]:
@benchmark btr(rosenbrock, ∇f, Hr, TruncatedCG, x, st)

BenchmarkTools.Trial: 
  memory estimate:  1.45 MiB
  allocs estimate:  3525
  --------------
  minimum time:     1.969 ms (0.00% GC)
  median time:      3.310 ms (0.00% GC)
  mean time:        4.179 ms (2.48% GC)
  maximum time:     48.042 ms (0.00% GC)
  --------------
  samples:          1189
  evals/sample:     1

In [32]:
@benchmark btr(rosenbrock, ∇f, Hess, TruncatedCG, x, st)

BenchmarkTools.Trial: 
  memory estimate:  722.61 KiB
  allocs estimate:  3225
  --------------
  minimum time:     611.300 μs (0.00% GC)
  median time:      836.101 μs (0.00% GC)
  mean time:        1.029 ms (2.39% GC)
  maximum time:     8.208 ms (0.00% GC)
  --------------
  samples:          4848
  evals/sample:     1

In [33]:
@benchmark btr(rosenbrock, ∇f, TriHess, TruncatedCG, x, st)

BenchmarkTools.Trial: 
  memory estimate:  667.89 KiB
  allocs estimate:  3331
  --------------
  minimum time:     526.600 μs (0.00% GC)
  median time:      704.900 μs (0.00% GC)
  mean time:        915.076 μs (2.21% GC)
  maximum time:     31.373 ms (0.00% GC)
  --------------
  samples:          5453
  evals/sample:     1

In [ ]:
#compute the second derivative of rosenbrook at x in the direction v
function gvr(x:: AbstractVector{T}) where T
    n = length(x)
    t = (-200*(x[2]^2-x[1])+2*(x[1]-1))*v[1]
    for i = 2:n-1
        t += (-200*(x[i+1]^2-x[i])+2*(x[i]-1)+400*x[i]*(x[i]^2-x[i-1]))*v[i]
    end
    t += 400*x[n]*(x[n]^2-x[n-1])*v[n]
    return t
end

v = ones(n)

Hxv = x -> ForwardDiff.gradient(gvr, x)
norm(Hxv(x)-TriHess(x)*v)

In [ ]:
mutable struct NotYetComputedMatrix{T} <: AbstractMatrix{T}
    x::Vector{T}
    f::Function
    function NotYetComputedMatrix(f::Function, T::Type = Float64)
        n = new{T}()
        n.f = f
        return n
    end
end

function H!(x::Vector{T}, stack::NotYetComputedMatrix{T}) where T
    stack.x = x
    stack
end

import Base.*
function *(h::NotYetComputedMatrix{T}, v::AbstractVector{T}) where T
    return h.f(h.x, v)
end

import Base.show
show(io::IO, mine::MIME{Symbol("text/plain")}, x::NotYetComputedMatrix{T}) where T = show(io, "NYCM{$T}")

In [ ]:
NotYetComputedMatrix(Hxv)